# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

## 1. What test is appropriate for this problem? Does CLT apply?

_A hypothesis test for comparing populations proportions is appropriate._

_I do not believe the CLT applies in this case - I do not think that the data is necessarily independent. Many factors go into the hiring process. A candidate could be called and receive the job, and therefore the next candidate on the list - whether they are black or white - would not be called. Therefore the variable of being called is dependent on other candidates._

## 2. What are the null and alternate hypotheses?

_Null hypothesis: Black and white candidates are called equally. P1 = P2_

_Alternate hypothesis: Black and white candidates are not called equally. P1 != P2_

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [9]:
df = data[['race', 'call']]

In [12]:
df.describe(include='all')

,race,call
count,4870,4870.000000
unique,2,NaN
top,w,NaN
freq,2435,NaN
mean,NaN,0.080493
std,NaN,0.272079
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000


In [13]:
df.race.value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [29]:
df.head()
p = df.pivot_table(index=['race','call'], aggfunc=len)
# also could be done with df.groupby(['race','call']).size()
p

race  call
b     0.0     2278
      1.0      157
w     0.0     2200
      1.0      235
dtype: int64

## 3. Compute margin of error, confidence interval, and p-value.

In [34]:
bnotcalled = len(df[(df.race == 'b') & (df.call == 0.0)])
bcalled = len(df[(df.race == 'b') & (df.call == 1.0)])
wnotcalled = len(df[(df.race == 'w') & (df.call == 0.0)])
wcalled = len(df[(df.race == 'w') & (df.call == 1.0)])

In [40]:
# let p1 be white proportion, p2 be black proportion
p1 = float(wcalled / wnotcalled)
p2 = float(bcalled / bnotcalled)
np1 = len(df.race[df.race == 'w'])
np2 = len(df.race[df.race == 'b'])
print(p1, np1, p2, np2)

0.10681818181818181 2435 0.06892010535557506 2435


In [41]:
sigma2p1 = (p1 * (1 - p1))/np1
sigma2p2 = (p2 * (1 - p2))/np2
print(sigma2p1, sigma2p2)

3.918195394301424e-05 2.6353233853532445e-05


In [42]:
p1p2diff = p1 - p2
p1p2sigma2 = np.sqrt(sigma2p1 + sigma2p2)
print(p1p2diff, p1p2sigma2)

0.03789807646260675 0.00809538064556


_Let us use a 95% signficance level - this equals a z-statistic of 1.96_

In [43]:
marginerror = p1p2sigma2 * 1.96
marginerror

0.01586694606530235

_Margin of error is .016 - or 1.6%_

In [44]:
print(p1p2diff - marginerror, p1p2diff + marginerror)

0.0220311303973 0.0537650225279


_Confidence interval is .038 +/- .016, which equals (0.022, 0.054). Or equivalently:_

_(2.2%, 5.4%)_

In [45]:
zscore = p1p2diff / p1p2sigma2
zscore

4.681444656142391

_With a z-statistic of 4.68, the null hypothesis is rejected. The p-value will be near 0._

## 4. Write a story describing the statistical significance in the context or the original problem.
## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

_In the context of the original problem - is there a difference in the amount of interviews requested between white and black sounding names on their resume - there is a statistically significant difference between the white and black sounding names. White sounding names were found to receive more callbacks on average._

_That being said, this analysis ONLY took into account the name on the resume. It did not take into account any other factors - such as education, years experience, jobs held, honors, or any other factors that go into the decision of whether or not to request an interview for a specific candidate._

_This analysis does not imply that race (by basis of name) is the most important factor in callback success - race was the only factor that was analyzed._

_At the least, this analysis should be amemded to include other factors that would go into the interview request decision as stated above._